[View in Colaboratory](https://colab.research.google.com/github/bhagatsajan0073/Google-Collaboration-Code/blob/master/RND.ipynb)

In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 21065 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...


In [5]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p my_drive
!google-drive-ocamlfuse my_drive


In [1]:
print ('Files in Drive:')
!ls my_drive/deeplearning_dataset/POC/

Files in Drive:
object_classification_data.hdf5  RND.ipynb  saved_models


In [2]:
import h5py
import numpy as np
import pandas as pd
import keras as K
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
drive_path='my_drive/deeplearning_dataset/POC/'

In [0]:
file=h5py.File(drive_path+"/object_classification_data.hdf5")

In [0]:
# file['images'][0,:].sum(axis=0)
train_x,test_x,train_y,test_y=train_test_split(file['images'][:],file['class'][:],test_size=0.2,random_state=42)
valid_x,score_x,valid_y,score_y=train_test_split(test_x,test_y,test_size=0.5,random_state=42)

In [6]:
print(train_x.shape)
print(valid_x.shape)
print(score_x.shape)

(1600, 256, 512)
(200, 256, 512)
(200, 256, 512)


In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os,gc
from keras.models import load_model

In [0]:
batch_size = 16
num_classes = 5
epochs = 1000
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(drive_path, 'saved_models')
model_name = 'keras_object_trained_model.h5'

In [0]:
# y_train = keras.utils.to_categorical(train_lables, num_classes)
# y_test = keras.utils.to_categorical(test_labels, num_classes)
y_tr = keras.utils.to_categorical(train_y, num_classes)
y_valid = keras.utils.to_categorical(valid_y, num_classes)
y_score = keras.utils.to_categorical(score_y, num_classes)

In [0]:
img_rows, img_cols = 256, 512

In [0]:
x_train = train_x.reshape(train_x.shape[0], img_rows, img_cols, 1)
# x_test = test_x.reshape(test_x.shape[0], img_rows, img_cols, 1)
x_valid = valid_x.reshape(valid_x.shape[0], img_rows, img_cols, 1)
x_score = score_x.reshape(score_x.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [12]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_valid.shape[0], 'valid samples')
print(x_score.shape[0], 'score samples')
# print(x_test.shape[0], 'test samples')

x_train shape: (1600, 256, 512, 1)
1600 train samples
200 valid samples
200 score samples


In [0]:
# from numba import cuda
# # !pip install numba
# # cuda.select_device()
# cuda.close()

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# initiate RMSprop optimizer
# opt = keras.optimizers.adagrad(lr=0.01, decay=1e-5)

# Let's train the model using RMSprop
# model.compile(loss='categorical_crossentropy',
#               optimizer=opt,
#               metrics=['accuracy'])

# datagen = ImageDataGenerator(
#     featurewise_center=False,  # set input mean to 0 over the dataset
#     samplewise_center=False,  # set each sample mean to 0
#     featurewise_std_normalization=False,  # divide inputs by std of the dataset
#     samplewise_std_normalization=False,  # divide each input by its std
#     zca_whitening=False,  # apply ZCA whitening
#     rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
#     width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#     height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#     horizontal_flip=True,  # randomly flip images
#     vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
# datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
# model.fit_generator(datagen.flow(x_train, y_tr,
#                                  batch_size=batch_size),
#                     epochs=epochs,
#                     validation_data=(x_valid, y_valid),
#                     workers=4)

model_path = os.path.join(save_dir, model_name)
model = load_model(model_path)

model.fit(x_train, y_tr,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_valid, y_valid))



Train on 1600 samples, validate on 200 samples
Epoch 1/1000
1024/1600 [==================>...........] - ETA: 30s - loss: 0.0052 - acc: 0.9971

In [15]:
score = model.evaluate(x_score, y_score, verbose=0)
valid_score=model.evaluate(x_valid, y_valid, verbose=0)

print('validation loss:', valid_score[0])s
print('validation accuracy:', valid_score[1])

print('score loss:', score[0])
print('score accuracy:', score[1])

validation loss: 0.1496699813749001
validation accuracy: 0.95
score loss: 0.4022110386379063
score accuracy: 0.94


In [20]:
# score
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at my_drive/deeplearning_dataset/POC/saved_models/keras_object_trained_model.h5 


In [0]:
from sklearn.metrics import confusion_matrix
pred=model.predict_classes(x_score)

In [34]:
# labels=['Words_400','Motorbikes','Faces','airplanes','BACKGROUND_Google']
confusion_matrix(np.argmax(y_score,axis=1),pred)

array([[40,  0,  0,  0,  1],
       [ 0, 40,  0,  0,  1],
       [ 0,  0, 30,  0,  2],
       [ 0,  0,  0, 36,  1],
       [ 0,  2,  4,  1, 42]])